In [ ]:
pip install -U ckiptagger[tf,gdown]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [9]:
from ckiptagger import data_utils, WS
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
1.88GB [00:11, 159MB/s]


In [2]:
df_train = pd.read_csv('/content/drive/MyDrive/NLP/news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('/content/drive/MyDrive/NLP/news_clustering_test.tsv', sep='\t')

In [3]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [4]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [5]:
ws = WS('./data/')
pos = POS('./data/')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [6]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [7]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

# 尋找降維的詞向量：PPMI + SVD

In [8]:
word2index = {}
index2word = {}
n = 0
for index in train_title_cuts:
    for word, flag in train_title_cuts[index]:
        if word in word2index:
            continue
        word2index[word] = n 
        index2word[n] = word
        n += 1

In [9]:
len(word2index)

6690

如果使用one-hot就需要這麼大的維度的詞向量

In [10]:
train_title_cuts[0]

[('亞洲杯', 'Nb'),
 ('奪', 'VC'),
 ('冠賠率', 'Na'),
 ('：', 'COLONCATEGORY'),
 ('日本', 'Nc'),
 ('、', 'PAUSECATEGORY'),
 ('伊朗', 'Nc'),
 ('領銜', 'VB'),
 (' ', 'WHITESPACE'),
 ('中國', 'Nc'),
 ('竟', 'D'),
 ('與', 'P'),
 ('泰國', 'Nc'),
 ('並列', 'VG')]

In [11]:
# 建立Co-Matrix
window_size = 1
def create_co_matrix(corpus):
    vocab_size = len(word2index)
    co_matrix = np.zeros(shape=(vocab_size, vocab_size), dtype=np.int32)
    for pairs in corpus.values():

        for sen_idx, word in enumerate(pairs):

            left_id = sen_idx-window_size if sen_idx-window_size>0 else 0
            right_id = sen_idx+window_size if sen_idx+window_size<len(pairs) else len(pairs)-1
            plus = pairs[left_id:sen_idx]+pairs[sen_idx+1:right_id+1]

            for plus_loc in plus:

                co_matrix[word2index[word[0]],word2index[plus_loc[0]]]+=1
    return co_matrix

In [12]:
co_matrix = create_co_matrix(train_title_cuts)

In [13]:
# 建立PPMI

def get_ppmi(co_matrix: np.ndarray, eps: float=1e-8):
    M = np.zeros_like(co_matrix,dtype=np.float32)
    N = np.sum(co_matrix)
    S = np.sum(co_matrix,axis=0)
    for i in range(co_matrix.shape[0]):
        for j in range(co_matrix.shape[1]):
            x = (co_matrix[i,j]*N)/(S[i]*S[j]+eps)
            pmi = np.log2(x) if x > 0 else 0
            M[i,j] = max(pmi,0)

    return M

ppmi = get_ppmi(co_matrix)

In [14]:
ppmi

array([[ 0.      ,  9.267157,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 9.267157,  0.      , 11.515084, ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      , 11.515084,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [15]:
# 進行SVD分解，並得到降維的詞向量

from sklearn.decomposition import TruncatedSVD

# 使用`TruncatedSVD`進行降維，降維到dim=1000
svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=42)
word_vectors = svd.fit_transform(ppmi)


In [16]:
word_vectors.shape

(6690, 1000)

# 新的詞向量 + Group mean vector: 測試

In [17]:
excluded_flags = [
    'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
    'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
    'Caa', 'Cab', 'Cba', 'Cbb',
    'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
    'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
    'SPCHANGECATEGORY', 'WHITESPACE'
]

In [18]:
train_svd_vectors = {}
for index, pairs in train_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    train_svd_vectors[index] = vector
    

In [19]:
test_svd_vectors = {}
for index, pairs in test_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    test_svd_vectors[index] = vector

In [20]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(train_svd_vectors.items()):
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)

In [21]:
def cosine_similarity(bow1, bow2):
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    return np.sum(bow1 * bow2) / (len_bow1 * len_bow2)

In [22]:
classification = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(test_svd_vectors.items()):
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [23]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 61, '遊戲': 10, '旅遊': 9, '財經': 8, '科技': 5, '農業': 4})
predict 財經 :  Counter({'財經': 62, '科技': 25, '農業': 16, '體育': 8, '遊戲': 8, '旅遊': 7})
predict 科技 :  Counter({'科技': 51, '體育': 15, '財經': 14, '遊戲': 10, '農業': 9, '旅遊': 8})
predict 旅遊 :  Counter({'旅遊': 58, '農業': 11, '科技': 5, '財經': 4, '體育': 2, '遊戲': 2})
predict 農業 :  Counter({'農業': 58, '旅遊': 7, '體育': 4, '遊戲': 4, '財經': 3, '科技': 2})
predict 遊戲 :  Counter({'遊戲': 66, '科技': 11, '財經': 9, '旅遊': 9, '體育': 8, '農業': 1})
